In [ ]:
import numpy as np
import utility_b as U

In [ ]:
# load pointcloud
vertices = np.load('../pointcloud/gt_vertices.npy')
colors = np.load('../pointcloud/gt_colors.npy')

Αλγόριθμος RANSAC

1. Επίλεξε τυχαία τον ελάχιστον αριθμό σημείων που απαιτοούνται για τον προσδιορισμό των παραμέτρων του μοντέλου.
2. Λύσε το πρόβλημα με τις παραμέτρους του μοντέλου.
3. Προσδιόρισε πόσα σημεία από το σύνολο όλων των σημείων ταιριάζουν με μια προκαθορισμένη ανοχή.
4. Εαν το κλάσμα του πλήθος των outliers δια το πλήθος όλων των σημείων υπερβαίνει ένα προκαθορισμένο όριο t, επίλεξε νέες παραμέτρους βάση των inliers.
5. Διαφορετικά, επανέλαβε τα βήματα 1 - 4 (το πολύ Ν φορές)

inliers -> σημεία του plane που θέλουμε να βρούμε ώστε να τα αφαιρέσουμε
outlier -> σημεία που δεν ανήκουν σε κάποιο plane

In [ ]:
# Example usage:
# Assuming you have a point cloud represented as a numpy array 'point_cloud' with shape (N, 3)
n_iterations = 10000
# threashold for graound trouth will be 150
# threashold for my pointcloud will be 20

threshold_distance = 150  # Define a threshold distance to consider a point as an inlier

total_inlier_points = np.array([])
total_inlier_colors = np.array([])
total_outlier_points = np.array([])
total_outlier_colors = np.array([])

# Run the RANSAC algorithm
for i in range(2):

    best_plane, best_inliers = U.ransac_plane_fit(vertices, n_iterations, threshold_distance)

    # Access the best fit plane coefficients
    a, b, c, d = best_plane

    # Access the inlier points from the point cloud
    inlier_points = vertices[best_inliers]
    inlier_colors = colors[best_inliers]
    print("inlier points shape: ", inlier_points.shape)
    print("inlier colors shape: ", inlier_colors.shape)

    # Access the outlier points from the point cloud
    outliers_indecies = np.delete(np.arange(vertices.shape[0]), best_inliers)
    outlier_points = vertices[outliers_indecies]
    outlier_colors = colors[outliers_indecies]
    print("outlier points shape: ", outlier_points.shape)
    print("outlier colors shape: ", outlier_colors.shape)

    # add points to total points
    total_inlier_points = np.append(total_inlier_points, inlier_points)
    total_inlier_colors = np.append(total_inlier_colors, inlier_colors)
    total_outlier_points = np.append(total_outlier_points, outlier_points)
    total_outlier_colors = np.append(total_outlier_colors, outlier_colors)

    vertices = outlier_points
    colors = outlier_colors

# save inlier points and colors
np.save('../pointcloud/inlier_vertices.npy', total_inlier_points)
np.save('../pointcloud/inlier_colors.npy', total_inlier_colors)

# save outlier points and colors
np.save('../pointcloud/outlier_vertices.npy', total_outlier_points)
np.save('../pointcloud/outlier_colors.npy', total_outlier_colors)
